<a href="https://colab.research.google.com/github/jrebull/AnaliticaPrescriptiva/blob/main/MIAAD_Progra_Rebull_Farm_Planning_Problem_vDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Eficiencia de Franquicias con DEA

**Materia:** Programación para Analítica Prescriptiva y de Apoyo a la Decisión  
**Alumno:** Javier Augusto Rebull Saucedo  
**Matrícula:** al263483

## Objetivo

Este proyecto utiliza el **Análisis Envolvente de Datos (DEA)** para evaluar la eficiencia productiva de 28 franquicias de automóviles. Se busca identificar qué franquicias operan de manera óptima (eficientes) y cuáles tienen áreas de mejora (ineficientes), basándose en sus recursos (inputs) y sus resultados (outputs).

El modelo se implementa en Python utilizando la librería `gurobipy` para resolver el problema de programación lineal subyacente.

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 94.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## Datos del Problema

Se definen los datos para las 28 franquicias, a las que llamaremos **Unidades de Toma de Decisión** (DMUs). Para cada una, se especifican sus inputs (recursos) y outputs (resultados).

* **Inputs (Recursos):**
    * `staff`: Número de empleados.
    * `showRoom`: Espacio de exhibición (en $100 m^2$).
    * `Population1`: Población en área de influencia tipo 1 (en 1000s).
    * `Population2`: Población en área de influencia tipo 2 (en 1000s).
    * `alphaEnquiries`: Consultas sobre el modelo Alpha (en 100s).
    * `betaEnquiries`: Consultas sobre el modelo Beta (en 100s).
* **Outputs (Resultados):**
    * `alphaSales`: Ventas del modelo Alpha (en 1000s).
    * `BetaSales`: Ventas del modelo Beta (en 1000s).
    * `profit`: Ganancia (en millones).

Estos datos se almacenan en una estructura `multidict` de Gurobi para facilitar su acceso en el modelo.

In [3]:
# Nombres de los inputs y outputs para referencia
inattr = ['staff', 'showRoom', 'Population1', 'Population2', 'alphaEnquiries', 'betaEnquiries']
outattr = ['alphaSales', 'BetaSales', 'profit']

# Datos de las 28 DMUs (franquicias)
dmus, inputs, outputs = gp.multidict({
    'Winchester': [{'staff': 7, 'showRoom': 8, 'Population1': 10, 'Population2': 12, 'alphaEnquiries': 8.5, 'betaEnquiries': 4}, {'alphaSales': 2, 'BetaSales': 0.6, 'profit': 1.5}],
    'Andover': [{'staff': 6, 'showRoom': 6, 'Population1': 20, 'Population2': 30, 'alphaEnquiries': 9, 'betaEnquiries': 4.5}, {'alphaSales': 2.3, 'BetaSales': 0.7, 'profit': 1.6}],
    'Basingstoke': [{'staff': 2, 'showRoom': 3, 'Population1': 40, 'Population2': 40, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.25, 'profit': 0.5}],
    'Poole': [{'staff': 14, 'showRoom': 9, 'Population1': 20, 'Population2': 25, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.6, 'BetaSales': 0.86, 'profit': 1.9}],
    'Woking': [{'staff': 10, 'showRoom': 9, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 11, 'betaEnquiries': 5}, {'alphaSales': 2.4, 'BetaSales': 1, 'profit': 2}],
    'Newbury': [{'staff': 24, 'showRoom': 15, 'Population1': 15, 'Population2': 13, 'alphaEnquiries': 25, 'betaEnquiries': 1.9}, {'alphaSales': 8, 'BetaSales': 2.6, 'profit': 4.5}],
    'Portsmouth': [{'staff': 6, 'showRoom': 7, 'Population1': 50, 'Population2': 40, 'alphaEnquiries': 8.5, 'betaEnquiries': 3}, {'alphaSales': 2.5, 'BetaSales': 0.9, 'profit': 1.6}],
    'Alresford': [{'staff': 8, 'showRoom': 7.5, 'Population1': 5, 'Population2': 8, 'alphaEnquiries': 9, 'betaEnquiries': 4}, {'alphaSales': 2.1, 'BetaSales': 0.85, 'profit': 2}],
    'Salisbury': [{'staff': 5, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 5, 'betaEnquiries': 2.5}, {'alphaSales': 2, 'BetaSales': 0.65, 'profit': 0.9}],
    'Guildford': [{'staff': 8, 'showRoom': 10, 'Population1': 30, 'Population2': 35, 'alphaEnquiries': 9.5, 'betaEnquiries': 4.5}, {'alphaSales': 2.05, 'BetaSales': 0.75, 'profit': 1.7}],
    'Alton': [{'staff': 7, 'showRoom': 8, 'Population1': 7, 'Population2': 8, 'alphaEnquiries': 3, 'betaEnquiries': 2}, {'alphaSales': 1.9, 'BetaSales': 0.70, 'profit': 0.5}],
    'Weybridge': [{'staff': 5, 'showRoom': 6.5, 'Population1': 9, 'Population2': 12, 'alphaEnquiries': 8, 'betaEnquiries': 4.5}, {'alphaSales': 1.8, 'BetaSales': 0.63, 'profit': 1.4}],
    'Dorchester': [{'staff': 6, 'showRoom': 7.5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 4}, {'alphaSales': 1.5, 'BetaSales': 0.45, 'profit': 1.45}],
    'Bridport': [{'staff': 11, 'showRoom': 8, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.2, 'BetaSales': 0.65, 'profit': 2.2}],
    'Weymouth': [{'staff': 4, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.8, 'BetaSales': 0.62, 'profit': 1.6}],
    'Portland': [{'staff': 3, 'showRoom': 3.5, 'Population1': 3, 'Population2': 20, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.9, 'BetaSales': 0.35, 'profit': 0.5}],
    'Chichester': [{'staff': 5, 'showRoom': 5.5, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 7, 'betaEnquiries': 3.5}, {'alphaSales': 1.2, 'BetaSales': 0.45, 'profit': 1.3}],
    'Petersfield': [{'staff': 21, 'showRoom': 12, 'Population1': 6, 'Population2': 6, 'alphaEnquiries': 15, 'betaEnquiries': 8}, {'alphaSales': 6, 'BetaSales': 0.25, 'profit': 2.9}],
    'Petworth': [{'staff': 6, 'showRoom': 5.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 8, 'betaEnquiries': 5}, {'alphaSales': 1.5, 'BetaSales': 0.55, 'profit': 1.55}],
    'Midhurst': [{'staff': 3, 'showRoom': 3.6, 'Population1': 3, 'Population2': 3, 'alphaEnquiries': 2.5, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.20, 'profit': 0.45}],
    'Reading': [{'staff': 30, 'showRoom': 29, 'Population1': 120, 'Population2': 80, 'alphaEnquiries': 35, 'betaEnquiries': 20}, {'alphaSales': 7, 'BetaSales': 2.5, 'profit': 8}],
    'Southampton': [{'staff': 25, 'showRoom': 16, 'Population1': 110, 'Population2': 80, 'alphaEnquiries': 27, 'betaEnquiries': 12}, {'alphaSales': 6.5, 'BetaSales': 3.5, 'profit': 5.4}],
    'Bournemouth': [{'staff': 19, 'showRoom': 10, 'Population1': 90, 'Population2': 22, 'alphaEnquiries': 25, 'betaEnquiries': 13}, {'alphaSales': 5.5, 'BetaSales': 3.1, 'profit': 4.5}],
    'Henley': [{'staff': 7, 'showRoom': 6, 'Population1': 5, 'Population2': 7, 'alphaEnquiries': 8.5, 'betaEnquiries': 4.5}, {'alphaSales': 1.2, 'BetaSales': 0.48, 'profit': 2}],
    'Maidenhead': [{'staff': 12, 'showRoom': 8, 'Population1': 7, 'Population2': 10, 'alphaEnquiries': 12, 'betaEnquiries': 7}, {'alphaSales': 4.5, 'BetaSales': 2, 'profit': 2.3}],
    'Fareham': [{'staff': 4, 'showRoom': 6, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.1, 'BetaSales': 0.48, 'profit': 1.7}],
    'Romsey': [{'staff': 2, 'showRoom': 2.5, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 2.5, 'betaEnquiries': 1}, {'alphaSales': 0.4, 'BetaSales': 0.1, 'profit': 0.55}],
    'Ringwood': [{'staff': 2, 'showRoom': 3.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 1.9, 'betaEnquiries': 1.2}, {'alphaSales': 0.3, 'BetaSales': 0.09, 'profit': 0.4}]
})

## Construcción del Modelo de Optimización

Para cada franquicia (DMU) que queramos evaluar (la llamaremos `target`), debemos resolver un problema de programación lineal.

* **Variables de Decisión:**
    * `wout[r]`: Peso o importancia asignada al output `r`.
    * `win[i]`: Peso o importancia asignada al input `i`.

* **Función Objetivo:**
    Maximizar la suma ponderada de los outputs de la franquicia `target`.
    $$ \text{Maximizar} \quad E_k = \sum_{r \in \text{Outputs}} \text{outvalue}_{r,k} \cdot u_{r} $$

* **Restricciones:**
    1.  **Normalización:** La suma ponderada de los inputs de la franquicia `target` debe ser igual a 1. Esto convierte el problema fraccional original en uno lineal.
        $$ \sum_{i \in \text{Inputs}} \text{invalue}_{i,k} \cdot v_{i} = 1 $$
    2.  **Eficiencia Relativa:** Para *cada* franquicia en el conjunto de datos (incluida la `target`), la suma ponderada de sus outputs no puede superar la suma ponderada de sus inputs. Esto asegura que ninguna franquicia tenga un ratio de eficiencia mayor a 1 con los pesos calculados.
        $$ \sum_{r \in \text{Outputs}} \text{outvalue}_{r,j} \cdot u_{r} - \sum_{i \in \text{Inputs}} \text{invalue}_{i,j} \cdot v_{i} \leq 0 \quad \forall j \in \text{DMUS} $$

La siguiente función `solve_DEA` encapsula la creación y resolución de este modelo para una franquicia `target` específica.

In [4]:
def solve_DEA(target_dmu, verbose=False):
    """
    Construye y resuelve el modelo de programación lineal DEA para una DMU objetivo.

    Args:
        target_dmu (str): El nombre de la DMU a evaluar.
        verbose (bool): Si es True, muestra el log del solver. Si es False, lo oculta.

    Returns:
        float: El score de eficiencia de la DMU objetivo.
    """
    # Crear el modelo
    model = gp.Model('DEA')

    # Variables de decisión: pesos para inputs y outputs
    wout = model.addVars(outattr, name="outputWeight")
    win = model.addVars(inattr, name="inputWeight")

    # Restricción 1: Normalización de inputs para la DMU objetivo
    model.addConstr((gp.quicksum(inputs[target_dmu][i] * win[i] for i in inattr) == 1), name='normalization')

    # Restricción 2: Ratios de eficiencia para TODAS las DMUs
    model.addConstrs((gp.quicksum(outputs[j][r] * wout[r] for r in outattr) -
                      gp.quicksum(inputs[j][i] * win[i] for i in inattr) <= 0
                      for j in dmus), name='ratios')

    # Función Objetivo: Maximizar la suma ponderada de outputs de la DMU objetivo
    model.setObjective(gp.quicksum(outputs[target_dmu][r] * wout[r] for r in outattr), GRB.MAXIMIZE)

    # Configurar el solver para no imprimir logs si verbose es False
    if not verbose:
        model.params.OutputFlag = 0

    # Resolver el modelo
    model.optimize()

    # Retornar el valor óptimo de la función objetivo (el score de eficiencia)
    return model.objVal

## Cálculo de la Eficiencia para Todas las Franquicias

Ahora, iteramos sobre la lista de todas las franquicias. En cada iteración, llamamos a la función `solve_DEA` para resolver el problema de optimización con esa franquicia como `target`.

Los scores de eficiencia resultantes se almacenan en un diccionario llamado `performance`.

In [5]:
# Diccionario para almacenar los resultados de eficiencia
performance = {}

# Resolver el modelo DEA para cada franquicia
for dmu in dmus:
    efficiency_score = solve_DEA(dmu, verbose=False)
    performance[dmu] = efficiency_score

print("Cálculo de eficiencia completado para todas las 28 franquicias.")

Restricted license - for non-production use only - expires 2026-11-23
Cálculo de eficiencia completado para todas las 28 franquicias.


## Análisis e Interpretación de Resultados

Una vez calculados todos los scores, procedemos a interpretarlos:

* **DMU Eficiente:** Si su score de eficiencia es igual a 1 (o muy cercano, debido a tolerancias numéricas). Esto significa que la franquicia opera en la "frontera de eficiencia". Ninguna otra franquicia (o combinación de ellas) puede producir más outputs con los mismos o menos inputs. Son los puntos de referencia del grupo.
* **DMU Ineficiente:** Si su score es menor que 1. El valor del score indica el grado de ineficiencia. Por ejemplo, un score de 0.85 sugiere que la franquicia podría, teóricamente, producir el mismo nivel de outputs utilizando solo el 85% de sus inputs actuales para alcanzar la frontera de eficiencia.

A continuación, clasificamos las franquicias en dos grupos y las ordenamos para una mejor visualización.

In [6]:
# Ordenar las franquicias por su score de eficiencia de mayor a menor
sorted_performance = {k: v for k, v in sorted(performance.items(), key=lambda item: item[1], reverse=True)}

# Listas para clasificar DMUs
efficient_dmus = []
inefficient_dmus = []

# Clasificar cada DMU
for dmu, score in sorted_performance.items():
    if score >= 0.99999:  # Usamos una tolerancia para la comparación de punto flotante
        efficient_dmus.append((dmu, score))
    else:
        inefficient_dmus.append((dmu, score))

# Imprimir los resultados de forma clara y ordenada
print("="*50)
print("              ANÁLISIS DE EFICIENCIA (DEA)")
print("="*50)

print("\n--- ✅ FRANQUICIAS EFICIENTES (Score = 1.0) ---")
print("Estas franquicias son los benchmarks del grupo.\n")
for dmu, score in efficient_dmus:
    print(f"  - {dmu:<15} | Score: {score:.3f}")

print("\n--- ❌ FRANQUICIAS INEFICIENTES (Score < 1.0) ---")
print("Estas franquicias tienen potencial de mejora.\n")
for dmu, score in inefficient_dmus:
    print(f"  - {dmu:<15} | Score: {score:.3f}")

print("\n" + "="*50)

              ANÁLISIS DE EFICIENCIA (DEA)

--- ✅ FRANQUICIAS EFICIENTES (Score = 1.0) ---
Estas franquicias son los benchmarks del grupo.

  - Newbury         | Score: 1.000
  - Alresford       | Score: 1.000
  - Salisbury       | Score: 1.000
  - Alton           | Score: 1.000
  - Weymouth        | Score: 1.000
  - Petersfield     | Score: 1.000
  - Southampton     | Score: 1.000
  - Bournemouth     | Score: 1.000
  - Maidenhead      | Score: 1.000
  - Fareham         | Score: 1.000
  - Romsey          | Score: 1.000
  - Basingstoke     | Score: 1.000
  - Portsmouth      | Score: 1.000
  - Portland        | Score: 1.000
  - Henley          | Score: 1.000

--- ❌ FRANQUICIAS INEFICIENTES (Score < 1.0) ---
Estas franquicias tienen potencial de mejora.

  - Petworth        | Score: 0.988
  - Reading         | Score: 0.984
  - Bridport        | Score: 0.982
  - Andover         | Score: 0.917
  - Ringwood        | Score: 0.908
  - Midhurst        | Score: 0.889
  - Dorchester      | Score:

### Nota sobre la Elección de Herramientas (Gurobi vs. Pyomo)

Para este problema, se utilizó la API nativa de **Gurobi** (`gurobipy`), que es muy eficiente para problemas de programación lineal.

Una alternativa es usar **Pyomo**, un lenguaje de modelado en Python. Pyomo ofrece mayor flexibilidad para cambiar de solver (por ejemplo, a `IPOPT` para problemas no lineales, o `GLPK` que es de código abierto).

* **Gurobi (nativo):** Ideal por su altísimo rendimiento y sintaxis directa cuando se sabe que el solver a usar es Gurobi.
* **Pyomo:** Excelente para la portabilidad del modelo y para problemas que podrían requerir diferentes tipos de solvers.

Dado que el DEA se formula como un problema de programación lineal, Gurobi es una elección óptima y directa.